# BirdFlock
Este ejemplo pretende simular el comportamiento de una bandada de pájaros: cientos, o incluso miles, de pájaros volando juntos, formando formas infinitas como si fueran una sola entidad. Para el desarrollo de este ejemplo, nos basaremos en el trabajo presentado por [Rohola Zandie](https://betterprogramming.pub/boids-simulating-birds-flock-behavior-in-python-9fff99375118).

## ¿Cómo funciona el modelo?
Una propuesta de solución a este problema fue presentado por Craig Reynolds cuando introdujo un sistema conocido como "Boids" que podía simular algo similar al comportamiento de bandadas de aves. Su modelo estable tres reglas simples:
* **Separación**: La separación es necesaria para que nuestros individuos choquen entre sí y se estrellen. Cada individuos debe ver a sus propios compañeros de bandada locales y alejarse si están demasiado cerca.
<center>
<img src="https://miro.medium.com/max/217/1*vcc16ijg_e8lMOTRIhhm9w.gif" />
</center>
* **Alineación**: Cada invidividuo solo ve a los individuos que están a su alrededor. Para la alineación, miramos los vecinos y calculamos su dirección promedio (que es parte del vector de velocidad) y seguimos eso.
<center>
<img src="https://miro.medium.com/max/217/1*e8AAGeLm9x0i6kkO4_Yenw.gif" />
</center>
* **Cohesión**: Significa dirigirse hacia el centro de masa de los compañeros de bandada locales. Hacemos esto para forzar a que los individuos se peguen entre sí y no se dividan.
<center>
<img src="https://miro.medium.com/max/217/1*Au6eWj6jETcbVAbQWJR1-g.gif" />
</center>

A diferencia de otros ejemplos, no queremos realizar una animación basada en una matriz sino en objetos que se mueven libremente en un espacio bidimencional. 

## Imports

In [1]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al azar''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(metric='euclidian')

In [ ]:
from mesa.model import Model

class Bird(Agent):
    def __init__(self, id, model, x, y, width, height):
        super().__init__(id, model)

        self.position = np.array((x,y), dtype=np.float64)

        # position
        vec = (np.random.rand(2) - 0.5) * 10
        
        # initial velocity of agent
        self.velocity = np.array(vec, dtype=np.float64)
        
        # initial acceleration of agent
        vec = (np.random.rand(2) - 0.5) / 2
        self.acceleration = np.array(vec, dtype=np.float64)

        # max turning force
        self.max_force = 0.3

        self.max_speed = 5

        # max distance that bird is looking at
        self.perception = 50

        self.width = width
        self.height = height
    
    def step(self):
        # each step will check the limits of the toroidal space
        self.check_edges()
        # each step will check the neighbors to change velocity and acceleration
        self.check_neighbors()

        self.position = self.position + self.velocity
        self.velocity = self.velocity + self.acceleration

        if np.linalg.norm(self.velocity) > self.max_speed:
            # lower speed in proportion to max_speed
            self.velocity = self.velocity / np.linalg.norm(self.velocity) * self.max_speed

        self.acceleration = np.array((0,0), dtype=np.float64)

    def check_edges(self):
        if self.position.flatten()[0] > self.width:
            self.position[0] = 0
        elif self.position.flatten()[0] < 0:
            self.position[0] = self.width
        
        if self.position.flatten()[1] > self.height:
            self.position[1] = 0
        elif self.position.flatten()[1] < 0:
            self.position[1] = self.height
    
    def check_neighbors(self):
        alignment = self.align()
        cohesion = self.cohesion()
        separation = self.separation()
        self.acceleration = self.acceleration + alignment
        self.acceleration = self.acceleration + cohesion
        self.acceleration = self.acceleration + separation
    
    def align(self):
        steering = np.array((0,0), dtype=np.float64)
        # avg alignment of neighbors
        avg_vector = np.array((0,0), dtype=np.float64)

        
        # return those neighboring agents within the perception radius 
        result = (neighbors.radius_neighbors([self.position], self.perception))
        for idx in result:
            avg_vector += model.schedule.agents[idx].velocity
        if len(result) > 0:
            avg_vector /= len(result)
            avg_vector = avg_vector / np.linalg.norm(avg_vector) * self.max_speed
            steering = avg_vector
        return steering

    def cohesion(self):
        steering = np.array((0,0), dtype=np.float64)
        center_of_mass = np.array((0,0), dtype=np.float64)

        # return those neighboring agents within the perception radius 
        result = (neighbors.radius_neighbors([self.position], self.perception))
        for idx in result:
            center_of_mass += model.schedule.agents[idx].position
        
        if len(result) > 0:
            center_of_mass /= len(result)
            vec_to_com = center_of_mass - self.position
            if np.linalg.norm(vec_to_com) > 0:
                vec_to_com = vec_to_com / np.linalg.norm(vec_to_com) * self.max_speed
            steering = vec_to_com - self.velocity
            if np.linalg.norm(steering)  > self.max_force:
                steering = steering / np.linalg.norm(steering) * self.max_force
                        
        return steering

    def separation(self):
        steering = np.array((0,0,) dtype=np.float64)
        avg_vector = np.array((0,0,) dtype=np.float64)

        result = (neighbors.radius_neighbors([self.position], self.perception))
        for idx in result:
            distance = self.position - model.schedule.agents[idx].position
            diff = distance / np.linalg.norm(distance)
            
            avg_vector += diff
            
        if len(result) > 0:
            avg_vector /= len(result)
            if np.linalg.norm(avg_vector) > 0:
                avg_vector = avg_vector / np.linalg.norm(avg_vector) * self.max_speed
            
            steering = avg_vector - self.velocity
            if np.linalg.norm(steering)  > self.max_force:
                steering = steering / np.linalg.norm(steering) * self.max_force

        return steering

In [ ]:
def get_particles(model):
    result = []
    for agent in model.schedule.agents:
        result.append(agent.position)
    result = np.asarray(result)
    return result

In [ ]:
class FlockModel(Model):
    def __init__(self, width, height, num_agents):
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(model_reporters={"Particles":get_particles})

        data=None
        for i in range(num_agents):
            x = np.random.rand() * width
            y = np.random.rand() * height
            agent = Bird((x,y), self, x,y,width,height)
            self.schedule.add(agent)
        
        # info for kd-tree
        # needs the position of all agents and adjusts their position
            if data is None:
                data= np.array([[x,y]])
            else:
                data = np.concatenate( (data, [[x,y]]) )
            # build kd tree
            neighbors.fit(data)
        
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
WIDTH = 1000
HEIGHT = 1000
NUM_AGENTS = 200
MAX_ITERATIONS = 2000

model = FlockModel(WIDTH,HEIGHT,NUM_AGENTS)
for i in range(MAX_ITERATIONS):
    model.step()